# 🗺️ Metapopulations
## Populations of Populations in Fragmented Landscapes

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/The-Pattern-Hunter/interactive-ecology-biometry/blob/main/unit-2-population/notebooks/05_metapopulations.ipynb)

---

> *"Think globally, act locally."* - René Dubos (adapted for metapopulations)

### 🎯 Learning Objectives

By the end of this notebook, you will:
1. Define **metapopulations** and their structure
2. Understand **patch occupancy dynamics**
3. Distinguish **source** vs **sink** populations
4. Model **colonization** and **extinction** rates
5. Analyze **rescue effects** and connectivity
6. Apply **island-mainland** and **Levins** models
7. Design conservation strategies for fragmented habitats

In [ ]:
# Setup
!pip install numpy pandas plotly matplotlib networkx scipy -q

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx

print("✅ Ready to explore metapopulations!")
print("🗺️ Let's understand spatial population dynamics!")

---

## 📚 Part 1: What is a Metapopulation?

### Definition:

**Metapopulation**: A "population of populations" - a set of local populations connected by dispersal

### Why Do Metapopulations Exist?

**Habitat fragmentation!**

#### Historical:
```
CONTINUOUS HABITAT          FRAGMENTED HABITAT
(Pre-development)           (Modern)

🌳🌳🌳🌳🌳🌳🌳              🌳🌳    🏢🏢    🌳
🌳🌳🌳🌳🌳🌳🌳              🌳🌳    🏢🏢    🌳
🌳🌳🌳🌳🌳🌳🌳    →         🏢🏢🏢🏢🏢🏢🏢
🌳🌳🌳🌳🌳🌳🌳              🏢🏢    🌳🌳🌳🌳
🌳🌳🌳🌳🌳🌳🌳              🏢🏢    🌳  🌳🌳

One large population        Multiple small patches
```

### Key Components:

#### 1. **Patches** (Habitat fragments)
- Discrete areas of suitable habitat
- Separated by unsuitable matrix
- Variable size and quality

#### 2. **Local Populations**
- Occupy individual patches
- Can go extinct locally
- Birth and death within patch

#### 3. **Dispersal**
- Movement between patches
- Colonizes empty patches
- Rescues declining populations

#### 4. **Matrix**
- Unsuitable habitat between patches
- Affects dispersal success
- Can be permeable or hostile

### Classic Metapopulation Assumptions:

✅ **All patches equal** (same size, quality)  
✅ **All populations equal** (same dynamics)  
✅ **Random extinction** (local pops can disappear)  
✅ **Random colonization** (empty patches can be recolonized)  
✅ **Global dispersal** (any patch can reach any other)  

**Reality**: These assumptions rarely hold, but model still useful!

### The Metapopulation Perspective:

**Traditional view**: Focus on single population size

**Metapopulation view**: Focus on:
- **Patch occupancy** (fraction of patches occupied)
- **Colonization rate** (new patches occupied)
- **Extinction rate** (patches losing populations)
- **Connectivity** (dispersal between patches)

### Why This Matters:

**Metapopulation can persist even when**:
- Individual populations go extinct
- No single patch suitable long-term
- Local disasters occur

**Key**: Colonization > Extinction (on average)

---

## 🎯 Part 2: The Levins Model - Classic Metapopulation

### Richard Levins (1969):

First mathematical model of metapopulations

### The Model:

**Focus**: Proportion of patches occupied (p)

#### **Change in occupancy**:
```
dp/dt = cp(1 - p) - ep
```

Where:
- **p** = proportion of patches occupied (0 to 1)
- **c** = colonization rate (new patches occupied per time)
- **e** = extinction rate (patches losing populations per time)
- **(1 - p)** = proportion of empty patches available

### Two Processes:

#### **1. Colonization** (+)
```
Colonization = cp(1 - p)
```
- Increases with occupied patches (p)
- Increases with empty patches (1 - p)
- Maximum at p = 0.5

#### **2. Extinction** (-)
```
Extinction = ep
```
- Proportional to occupied patches
- Constant rate per patch

### Equilibrium:

**At equilibrium** (dp/dt = 0):
```
p* = 1 - (e/c)
```

### Critical Insight:

**Metapopulation persists if**:
```
c > e
```

**Colonization must exceed extinction!**

### Three Scenarios:

#### **Scenario 1: c > e** (Persistence)
- p* = positive equilibrium
- Metapopulation persists
- Example: c = 0.4, e = 0.1 → p* = 0.75 (75% occupied)

#### **Scenario 2: c = e** (Threshold)
- p* = 0 (critical point)
- Tipping point
- Metapopulation on edge

#### **Scenario 3: c < e** (Extinction)
- p* = negative (impossible)
- All patches eventually empty
- Metapopulation goes extinct
- Example: c = 0.1, e = 0.4 → Extinction

In [ ]:
# Simulate Levins metapopulation model
def levins_model(p, t, c, e):
    """
    Levins metapopulation dynamics
    dp/dt = cp(1-p) - ep
    """
    dpdt = c * p * (1 - p) - e * p
    return dpdt

from scipy.integrate import odeint

# Time span
t = np.linspace(0, 100, 1000)

# Initial occupancy
p0 = 0.1  # Start with 10% patches occupied

# Three scenarios
scenarios = [
    {'name': 'High Colonization (c > e)', 'c': 0.4, 'e': 0.1, 'color': 'green'},
    {'name': 'Balanced (c ≈ e)', 'c': 0.25, 'e': 0.2, 'color': 'orange'},
    {'name': 'High Extinction (c < e)', 'c': 0.15, 'e': 0.3, 'color': 'red'}
]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        'Patch Occupancy Over Time',
        'Phase Diagram (Colonization vs Extinction)'
    ),
    horizontal_spacing=0.12
)

# Simulate each scenario
for scenario in scenarios:
    c, e = scenario['c'], scenario['e']
    p = odeint(levins_model, p0, t, args=(c, e)).flatten()
    
    # Equilibrium
    p_eq = max(0, 1 - e/c)
    
    # Time series
    fig.add_trace(
        go.Scatter(x=t, y=p,
                   mode='lines',
                   line=dict(width=3, color=scenario['color']),
                   name=f"{scenario['name']}<br>p* = {p_eq:.2f}"),
        row=1, col=1
    )

# Phase diagram
p_range = np.linspace(0, 1, 100)
for scenario in scenarios:
    c, e = scenario['c'], scenario['e']
    colonization = c * p_range * (1 - p_range)
    extinction = e * p_range
    
    fig.add_trace(
        go.Scatter(x=p_range, y=colonization,
                   mode='lines',
                   line=dict(width=2, color=scenario['color']),
                   name=f"Colonization (c={c})",
                   showlegend=False),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Scatter(x=p_range, y=extinction,
                   mode='lines',
                   line=dict(width=2, color=scenario['color'], dash='dash'),
                   name=f"Extinction (e={e})",
                   showlegend=False),
        row=1, col=2
    )

# Update axes
fig.update_xaxes(title_text="Time (years)", row=1, col=1)
fig.update_xaxes(title_text="Patch Occupancy (p)", row=1, col=2)
fig.update_yaxes(title_text="Proportion Occupied (p)", range=[0, 1], row=1, col=1)
fig.update_yaxes(title_text="Rate", row=1, col=2)

fig.update_layout(
    title="🎯 Levins Metapopulation Model<br><sub>dp/dt = cp(1-p) - ep</sub>",
    height=500,
    template='plotly_white'
)

fig.show()

print("\n🎯 Levins Model Results:\n")
for scenario in scenarios:
    c, e = scenario['c'], scenario['e']
    p_eq = max(0, 1 - e/c)
    
    print(f"   {scenario['name']}:")
    print(f"      • Colonization rate (c): {c}")
    print(f"      • Extinction rate (e): {e}")
    print(f"      • Ratio c/e: {c/e:.2f}")
    print(f"      • Equilibrium (p*): {p_eq:.2f} ({p_eq*100:.0f}% occupied)")
    
    if c > e:
        print(f"      • Outcome: PERSISTENCE ✅")
    elif c == e:
        print(f"      • Outcome: THRESHOLD ⚠️")
    else:
        print(f"      • Outcome: EXTINCTION ❌")
    print()

print("\n💡 Key Insights:")
print("   • Colonization must EXCEED extinction")
print("   • Even with local extinctions, metapop can persist")
print("   • Equilibrium depends on c/e ratio")
print("   • Never reaches 100% occupancy (always some turnover)")
print("\n📊 Phase Diagram:")
print("   • Where lines cross = equilibrium")
print("   • Solid line (colonization) hump-shaped")
print("   • Dashed line (extinction) straight")
print("   • Intersection determines p*")

---

## 🏝️ Part 3: Source-Sink Dynamics

### Not All Patches Are Equal!

Real metapopulations have **heterogeneous patches**

### Two Types of Populations:

#### **Source Populations** ✅

**Definition**: Birth rate > Death rate (b > d)

**Characteristics**:
- High-quality habitat
- Net reproductive output > 1
- Population GROWS if isolated
- **Produces emigrants**
- Can sustain themselves

**Examples**:
- Large, intact forest patches
- Protected reserves
- Optimal habitat

#### **Sink Populations** ❌

**Definition**: Birth rate < Death rate (b < d)

**Characteristics**:
- Poor-quality habitat
- Net reproductive output < 1
- Population DECLINES if isolated
- **Depends on immigrants**
- Cannot sustain themselves

**Examples**:
- Small habitat fragments
- Edge habitats
- Suboptimal conditions

### Source-Sink Dynamics:

```
SOURCE          →  Dispersal  →       SINK
🟢 High quality    (emigrants)       🔴 Low quality
b > d              -------→          b < d
Surplus production                   Depends on input
```

### Key Patterns:

#### **1. Source Subsidizes Sink**
- Emigrants from source keep sink occupied
- Sink would go extinct without immigration
- Creates "rescue effect"

#### **2. Sink Can Be "Attractive Trap"**
- Animals attracted to sink habitat
- But can't reproduce successfully
- Drains source population
- Example: Birds attracted to urban parks

#### **3. Spatial Distribution Misleading**
- Can't judge population health by presence
- Sink may appear occupied (but not self-sustaining)
- Need to measure reproduction, not just density

### Conservation Implications:

⚠️ **Must protect SOURCES first!**
- Sources are critical for metapopulation
- Sinks depend on sources
- Lose sources → Entire metapopulation collapses

✅ **Identify sources**:
- Measure reproduction (not just presence)
- Look for net emigration
- Assess habitat quality

🔄 **Sinks can be valuable**:
- Increase total population size
- Provide connectivity
- Can become sources if improved

In [ ]:
# Simulate source-sink dynamics
def source_sink_model(state, t, b_source, d_source, b_sink, d_sink, m):
    """
    Two-patch source-sink model
    state = [N_source, N_sink]
    m = migration rate (proportion dispersing)
    """
    N_source, N_sink = state
    
    # Net growth in each patch
    growth_source = (b_source - d_source) * N_source
    growth_sink = (b_sink - d_sink) * N_sink
    
    # Migration (source → sink)
    emigration = m * N_source
    
    # Change in each patch
    dN_source = growth_source - emigration
    dN_sink = growth_sink + emigration
    
    return [dN_source, dN_sink]

# Parameters
b_source = 0.5  # Birth rate in source
d_source = 0.3  # Death rate in source (b > d)
b_sink = 0.2    # Birth rate in sink
d_sink = 0.4    # Death rate in sink (b < d)
m = 0.1         # Migration rate (10% emigrate)

# Initial conditions
N_source0 = 100
N_sink0 = 20
state0 = [N_source0, N_sink0]

# Time span
t = np.linspace(0, 50, 500)

# Simulate with migration
solution = odeint(source_sink_model, state0, t, 
                  args=(b_source, d_source, b_sink, d_sink, m))
N_source, N_sink = solution[:, 0], solution[:, 1]

# Simulate sink in ISOLATION (no migration)
solution_isolated = odeint(source_sink_model, state0, t,
                           args=(b_source, d_source, b_sink, d_sink, 0))  # m=0
N_sink_isolated = solution_isolated[:, 1]

# Create visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        'Source and Sink Populations',
        'Sink: With vs Without Immigration'
    ),
    horizontal_spacing=0.12
)

# Source and sink
fig.add_trace(
    go.Scatter(x=t, y=N_source,
               mode='lines',
               line=dict(width=3, color='green'),
               fill='tozeroy',
               fillcolor='rgba(0,255,0,0.2)',
               name='Source (b > d)'),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=t, y=N_sink,
               mode='lines',
               line=dict(width=3, color='red'),
               fill='tozeroy',
               fillcolor='rgba(255,0,0,0.2)',
               name='Sink (b < d)'),
    row=1, col=1
)

# Sink comparison
fig.add_trace(
    go.Scatter(x=t, y=N_sink,
               mode='lines',
               line=dict(width=3, color='blue'),
               name='With Immigration',
               showlegend=False),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=t, y=N_sink_isolated,
               mode='lines',
               line=dict(width=3, color='orange', dash='dash'),
               name='Isolated (No Immigration)',
               showlegend=False),
    row=1, col=2
)

# Update axes
fig.update_xaxes(title_text="Time (years)")
fig.update_yaxes(title_text="Population Size")

fig.update_layout(
    title="🏝️ Source-Sink Metapopulation Dynamics<br><sub>Migration maintains sink population</sub>",
    height=500,
    template='plotly_white'
)

fig.show()

# Calculate intrinsic growth rates
r_source = b_source - d_source
r_sink = b_sink - d_sink

print("\n🏝️ Source-Sink Analysis:\n")
print("   SOURCE Population:")
print(f"      • Birth rate (b): {b_source}")
print(f"      • Death rate (d): {d_source}")
print(f"      • Net growth (r): {r_source:+.2f} (POSITIVE ✅)")
print(f"      • Final size: {N_source[-1]:.0f}")
print("      • Outcome: Grows even with emigration")
print("\n   SINK Population:")
print(f"      • Birth rate (b): {b_sink}")
print(f"      • Death rate (d): {d_sink}")
print(f"      • Net growth (r): {r_sink:+.2f} (NEGATIVE ❌)")
print(f"      • Final size (with immigration): {N_sink[-1]:.0f}")
print(f"      • Final size (isolated): {N_sink_isolated[-1]:.0f}")
print("\n💡 Key Observations:")
print("   • Source PRODUCES individuals (net positive growth)")
print("   • Sink CONSUMES individuals (net negative growth)")
print(f"   • Migration ({m*100:.0f}% emigration) maintains sink")
print("   • Without immigration, sink goes extinct!")
print("   • Source loses some individuals but still grows")
print("\n⚠️ Conservation Message:")
print("   • MUST protect source habitats!")
print("   • Sink presence doesn't indicate health")
print("   • Need connectivity for sink persistence")
print("   • Improve sinks → convert to sources")

---

## 🔗 Part 4: Connectivity & Rescue Effects

### Why Connectivity Matters:

**Dispersal is the lifeblood of metapopulations!**

### Three Benefits of Connectivity:

#### **1. Colonization** (Reestablishment)
- Empty patches can be recolonized
- Maintains metapopulation persistence
- Spreads risk across patches

#### **2. Rescue Effect** (Demographic support)
- Immigrants boost declining populations
- Prevents local extinction
- **Key**: Arrives BEFORE extinction

#### **3. Genetic Rescue** (Evolutionary support)
- New genes reduce inbreeding
- Increases genetic diversity
- Improves fitness

### Factors Affecting Connectivity:

#### **Distance**
- Closer patches → Higher connectivity
- Dispersal success declines with distance
- **Isolation** = low connectivity

#### **Matrix Quality**
- **Permeable**: Easy to cross (grassland between forests)
- **Hostile**: Difficult/deadly (highway, urban area)
- Affects effective distance

#### **Corridors**
- Linear habitat connecting patches
- Facilitate movement
- Reduce effective isolation
- **Examples**: Hedgerows, riparian strips, wildlife overpasses

#### **Stepping Stones**
- Small patches between large patches
- Intermediate stops for dispersal
- Increase connectivity without corridors

### Measuring Connectivity:

#### **Incidence Function**:
```
J_i = Σ exp(-αd_ij) × A_j
```

**Connectivity of patch i** (J_i) depends on:
- **d_ij**: Distance to other patches
- **α**: Distance decay parameter
- **A_j**: Area of source patches

**Higher J_i → More connected → Lower extinction risk**

### The Isolation Threshold:

**Too isolated → Cannot be rescued → Goes extinct**

```
Extinction Risk
  ↑
  |              /
  |            /
  |          /
  |        /
  |      /
  |____/______________→
    Near      Far
       Distance to Other Patches
```

In [ ]:
# Visualize connectivity and rescue effect
np.random.seed(42)

# Create a landscape with patches
n_patches = 15
patch_coords = np.random.rand(n_patches, 2) * 100  # Random locations
patch_areas = np.random.uniform(10, 50, n_patches)  # Random sizes
patch_occupied = np.random.choice([True, False], n_patches, p=[0.7, 0.3])  # 70% occupied

# Calculate connectivity for each patch
def calculate_connectivity(coords, areas, alpha=0.05):
    n = len(coords)
    connectivity = np.zeros(n)
    
    for i in range(n):
        for j in range(n):
            if i != j:
                distance = np.sqrt(np.sum((coords[i] - coords[j])**2))
                connectivity[i] += np.exp(-alpha * distance) * areas[j]
    
    return connectivity

connectivity = calculate_connectivity(patch_coords, patch_areas)

# Normalize for visualization
connectivity_norm = (connectivity - connectivity.min()) / (connectivity.max() - connectivity.min())

# Create network graph
G = nx.Graph()

# Add nodes
for i in range(n_patches):
    G.add_node(i, pos=patch_coords[i], 
               size=patch_areas[i],
               occupied=patch_occupied[i],
               connectivity=connectivity[i])

# Add edges for nearby patches (within threshold distance)
distance_threshold = 30
for i in range(n_patches):
    for j in range(i+1, n_patches):
        distance = np.sqrt(np.sum((patch_coords[i] - patch_coords[j])**2))
        if distance < distance_threshold:
            G.add_edge(i, j, weight=1/distance)

# Create visualization
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=(
        'Metapopulation Network (Connectivity)',
        'Connectivity vs Extinction Risk'
    ),
    horizontal_spacing=0.12,
    specs=[[{'type': 'scatter'}, {'type': 'scatter'}]]
)

# Network plot
# Draw edges
for edge in G.edges():
    x0, y0 = patch_coords[edge[0]]
    x1, y1 = patch_coords[edge[1]]
    fig.add_trace(
        go.Scatter(x=[x0, x1], y=[y0, y1],
                   mode='lines',
                   line=dict(width=0.5, color='lightgray'),
                   showlegend=False,
                   hoverinfo='skip'),
        row=1, col=1
    )

# Draw nodes (patches)
for i in range(n_patches):
    x, y = patch_coords[i]
    color = 'green' if patch_occupied[i] else 'lightgray'
    size = patch_areas[i] * 0.8
    
    fig.add_trace(
        go.Scatter(x=[x], y=[y],
                   mode='markers+text',
                   marker=dict(size=size, color=color, 
                              line=dict(width=2, color='black')),
                   text=f"{i}",
                   textposition='middle center',
                   name='Occupied' if patch_occupied[i] and i == 0 else '',
                   showlegend=(i == 0 or (i == 1 and not patch_occupied[0])),
                   hovertext=f"Patch {i}<br>Area: {patch_areas[i]:.0f}<br>" + 
                            f"Connectivity: {connectivity[i]:.1f}<br>" +
                            f"Status: {'Occupied' if patch_occupied[i] else 'Empty'}"),
        row=1, col=1
    )

# Connectivity vs extinction risk
extinction_risk = 0.5 * np.exp(-connectivity_norm * 3)  # Risk decreases with connectivity

fig.add_trace(
    go.Scatter(x=connectivity, y=extinction_risk,
               mode='markers',
               marker=dict(size=patch_areas*0.5,
                          color=['green' if occ else 'red' for occ in patch_occupied],
                          line=dict(width=1, color='black')),
               showlegend=False,
               hovertext=[f"Patch {i}<br>Connectivity: {connectivity[i]:.1f}<br>" +
                         f"Extinction Risk: {extinction_risk[i]:.2f}"
                         for i in range(n_patches)]),
    row=1, col=2
)

# Update axes
fig.update_xaxes(title_text="X Position (km)", showgrid=False, row=1, col=1)
fig.update_xaxes(title_text="Connectivity Index", row=1, col=2)
fig.update_yaxes(title_text="Y Position (km)", showgrid=False, row=1, col=1)
fig.update_yaxes(title_text="Extinction Risk", row=1, col=2)

fig.update_layout(
    title="🔗 Landscape Connectivity & Extinction Risk<br><sub>Green = Occupied | Gray = Empty | Lines = Potential Dispersal</sub>",
    height=600,
    template='plotly_white'
)

fig.show()

# Analysis
print("\n🔗 Connectivity Analysis:\n")
print(f"   Total patches: {n_patches}")
print(f"   Occupied: {patch_occupied.sum()} ({patch_occupied.sum()/n_patches*100:.0f}%)")
print(f"   Empty: {(~patch_occupied).sum()} ({(~patch_occupied).sum()/n_patches*100:.0f}%)")
print(f"\n   Connectivity range: {connectivity.min():.1f} - {connectivity.max():.1f}")
print(f"   Mean connectivity: {connectivity.mean():.1f}")
print(f"\n   Most connected patch: {connectivity.argmax()} (J = {connectivity.max():.1f})")
print(f"   Most isolated patch: {connectivity.argmin()} (J = {connectivity.min():.1f})")
print("\n💡 Patterns:")
print("   • Higher connectivity → Lower extinction risk")
print("   • Isolated patches more likely empty")
print("   • Central patches better connected")
print("   • Edge patches more vulnerable")
print("\n🛠️ Management Recommendations:")
print("   1. Increase connectivity (corridors, stepping stones)")
print("   2. Protect well-connected patches (hubs)")
print("   3. Improve matrix permeability")
print("   4. Prioritize patches with high connectivity")
print("   5. Consider landscape-scale conservation")

---

## 🎓 Summary

### Key Takeaways:

✅ **Metapopulation**: Population of populations connected by dispersal  
✅ **Levins model**: Focus on patch occupancy, not abundance  
✅ **c > e required**: Colonization must exceed extinction  
✅ **Source-sink**: Not all patches equal (sources critical)  
✅ **Rescue effect**: Immigration prevents local extinction  
✅ **Connectivity**: Dispersal is lifeblood of metapopulations  
✅ **Fragmentation**: Modern conservation challenge  

### The Levins Model:

**Equation**:
```
dp/dt = cp(1 - p) - ep
```

**Equilibrium**:
```
p* = 1 - (e/c)
```

**Persistence condition**:
```
c/e > 1
```

### Source vs Sink:

| Feature | Source | Sink |
|---------|--------|------|
| **Growth** | b > d (positive) | b < d (negative) |
| **Fate if isolated** | Persist | Go extinct |
| **Role** | Produce emigrants | Depend on immigrants |
| **Habitat** | High quality | Low quality |
| **Priority** | CRITICAL | Secondary |

### Connectivity Benefits:

1. **Colonization**: Reestablish extinct populations
2. **Rescue effect**: Save declining populations
3. **Genetic rescue**: Reduce inbreeding
4. **Risk spreading**: Not "all eggs in one basket"

### Conservation Strategies:

#### 🎯 **Patch-Level**:
- Protect large patches (lower extinction)
- Prioritize source habitats
- Improve sink quality → convert to sources
- Maintain habitat heterogeneity

#### 🔗 **Landscape-Level**:
- Create corridors between patches
- Add stepping-stone patches
- Improve matrix permeability
- Reduce barriers (roads, development)

#### 📊 **Metapopulation-Level**:
- Ensure c > e (overall)
- Maintain multiple occupied patches
- Monitor occupancy dynamics
- Prevent isolation

### Real-World Applications:

#### 🦋 **Butterflies**:
- Classic metapopulation system
- Patch turnover observed
- Connectivity critical
- Example: Checkerspot butterflies

#### 🐸 **Amphibians**:
- Pond metapopulations
- Source-sink dynamics common
- Terrestrial matrix important
- Example: Pool frogs

#### 🌳 **Forest Fragments**:
- Many species now in metapopulations
- Birds, mammals, plants
- Connectivity via corridors
- Example: Spotted owls

#### 🏞️ **Reserve Networks**:
- Design as metapopulations
- Multiple protected areas
- Connected by corridors
- Example: Yellowstone to Yukon

### Key Equations & Concepts:

**Metapopulation capacity**:
```
λ_M = Largest eigenvalue of colonization matrix
```
If λ_M > extinction rate → Persistence

**Connectivity index**:
```
J_i = Σ exp(-αd_ij) × A_j
```

**Extinction risk**:
```
E_i = e / (1 + J_i)
```
Higher connectivity → Lower extinction

### Limitations of Models:

**Assumptions often violated**:
- Patches not equal
- Dispersal not random
- Local dynamics matter
- Spatial structure important

**But still useful**:
- General principles hold
- Qualitative predictions robust
- Framework for thinking
- Guide management

### The Bottom Line:

**Modern landscapes are fragmented**
- Many species now in metapopulations
- Need landscape-scale thinking
- Connectivity is CRITICAL
- Single reserves insufficient

**Conservation must**:
- Protect multiple patches
- Maintain connectivity
- Prioritize sources
- Think spatially

---

<div align="center">

**Made with 💚 by The Pattern Hunter Team**

[📓 Previous: Population Regulation](04_population_regulation.ipynb) | 
[🏠 Unit 2 Home](../../)

**🎉 Congratulations! You've completed Unit 2: Population Ecology! 🎉**

</div>